# LeNet5
---
http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf
<img src="lenet5.png">

## Possible Hyperparameters
---
1. the number of convnet layers
2. the number of maxpooling layers followed by convnet
3. the depth of each convnet
4. the size of batch (if Stochastic Gradient Descent)
5. the size of patch for each convnet, and same patching or valid patching
6. the number of neurons in each fully connected layer
7. softmax (or hierarchical softmax)
8. constant learning rate (or exponential decay learning rate)

## Data Preprocess
---
1. setting each pose as one object (or ignoring the poses) 
2. box filter (or cropping on depth image)
3. adding noise (or shifting the objects)
4. training data and test data

## MNIST LeNet5

#### 1. Import Packages

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

#### 2. Load Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### 3. Pre-process 

In [18]:
image_size = 28
num_channels = 1 # greyscale

train_data = mnist.train.images
train_labels = mnist.train.labels
validation_data = mnist.validation.images
validation_labels = mnist.validation.labels
test_data = mnist.test.images
test_labels = mnist.test.labels

def reformat(data):
    reformated_data = data.reshape(-1, image_size, image_size, num_channels).astype(np.float32)
    return reformated_data

train_dataset = reformat(train_data)
validation_dataset = reformat(validation_data)
test_dataset = reformat(test_data)

# print out all data shapes
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', validation_dataset.shape, validation_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (55000, 28, 28, 1) (55000, 10)
Validation set (5000, 28, 28, 1) (5000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


#### 4. Define Costumed Function

In [22]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

#### 5. Build NN

In [20]:
import time
start_time = time.time()

batch_size = 16
patch_size = 5
kernel_size = 2
depth1 = 6 #the depth of 1st convnet
depth2 = 16 #the depth of 2nd convnet
C5_units = 120
F6_units = 84
F7_units = 10

graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    # convolution's input is a tensor of shape [batch,in_height,in_width,in_channels]
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, 10))
    tf_valid_dataset = tf.constant(validation_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables(weights and biases)
    C1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth1], stddev=0.1))
    # convolution's weights are called filter in tensorflow
    # it is a tensor of shape [kernel_hight,kernel_width,in_channels,out_channels]
    C1_biases = tf.Variable(tf.zeros([depth1]))
                            
    # S1_weights # Sub-sampling doesn't need weights and biases
    # S1_biases
    
    C3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth1, depth2], stddev=0.1))
    C3_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
                            
    # S4_weights
    # S4_biases
     
    # C5 actually is a fully-connected layer                        
    C5_weights = tf.Variable(tf.truncated_normal([5 * 5 * depth2, C5_units], stddev=0.1))
    C5_biases = tf.Variable(tf.constant(1.0, shape=[C5_units]))
         
    F6_weights = tf.Variable(tf.truncated_normal([C5_units,F6_units], stddev=0.1))
    F6_biases = tf.Variable(tf.constant(1.0, shape=[F6_units]))
                                
    # FC and logistic regression replace RBF
    F7_weights = tf.Variable(tf.truncated_normal([F6_units,F7_units], stddev=0.1))
    F7_biases = tf.Variable(tf.constant(1.0, shape=[F7_units]))

    # Model
    def model(data):
        conv = tf.nn.conv2d(data, C1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + C1_biases) # relu is better than tanh
        
        max_pool = tf.nn.max_pool(hidden,[1,kernel_size,kernel_size,1],[1,2,2,1],'VALID')
        hidden = tf.nn.relu(max_pool)
                                
        conv = tf.nn.conv2d(hidden, C3_weights, [1, 1, 1, 1], padding='VALID')
        hidden = tf.nn.relu(conv + C3_biases)

        max_pool = tf.nn.max_pool(hidden,[1,kernel_size,kernel_size,1],[1,2,2,1],'VALID')
        hidden = tf.nn.relu(max_pool)
                            
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, C5_weights) + C5_biases)
                            
        fc = tf.matmul(hidden,F6_weights)
        hidden = tf.nn.relu(fc + F6_biases)
        
        fc = tf.matmul(hidden,F7_weights)
        output = fc + F7_biases
    
        return output

    
    # Training computation.
    tf_train_dataset = tf.nn.dropout(tf_train_dataset,0.8) # input dropout
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

#### 6. Train NN

In [24]:
# training
num_steps = 5000
config = tf.ConfigProto()
config.log_device_placement = True
with tf.Session(graph=graph, config = config) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), validation_labels))
      print('--------------------------------------')
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
end_time = time.time()
duration = (end_time - start_time)/60
print("Excution time: %0.2fmin" % duration)

Initialized
Minibatch loss at step 0: 2.616367
Minibatch accuracy: 18.8%
Validation accuracy: 11.3%
--------------------------------------
Minibatch loss at step 500: 0.617377
Minibatch accuracy: 75.0%
Validation accuracy: 89.9%
--------------------------------------
Minibatch loss at step 1000: 0.045929
Minibatch accuracy: 100.0%
Validation accuracy: 94.3%
--------------------------------------
Minibatch loss at step 1500: 0.254517
Minibatch accuracy: 87.5%
Validation accuracy: 96.2%
--------------------------------------
Minibatch loss at step 2000: 0.028234
Minibatch accuracy: 100.0%
Validation accuracy: 96.9%
--------------------------------------
Minibatch loss at step 2500: 0.032960
Minibatch accuracy: 100.0%
Validation accuracy: 96.8%
--------------------------------------
Minibatch loss at step 3000: 0.010193
Minibatch accuracy: 100.0%
Validation accuracy: 97.2%
--------------------------------------
Minibatch loss at step 3500: 0.007643
Minibatch accuracy: 100.0%
Validation ac